# Graph Deep Copy

* Given a reference to a node in an undirected graph
* Create a deep copy of the graph
* The copy must be independant from the original
* New nodes must be created


<span style="color:orange"><b>The point:</b></span>

* Duplicate while traversing
* DFS



**Complexity :**

| Time        | Space        |
|-------------|--------------|
| O(n + e)    | O(n)         |

* O(n x capa) in time because we create n node and travers e edges
* O(n) in space because the size of the recursive stack (can grow up to n) + clone_map hash map stores n key-val pairs









<!-- <span style="color:red"><b>TODO : </b></span> 
* Add comments in code -->


<!-- * <span style="color:lime"><b>Preferred solution?</b></span>      -->



## V1

**About Rust :**
* There are no pointer relationships, so deep copy can be very simple.
* **YES** : tested on the [Rust Playground](https://play.rust-lang.org/)


In [ ]:
use std::collections::HashSet;

struct Graph {
    nodes: Vec<GraphNode>,
}

impl Graph {
    fn new() -> Self {
        Self { nodes: Vec::new() }
    }

    fn from_adjacency_list(adj_list: &[Vec<usize>]) -> Self {
        let mut graph = Graph::new();
        for (i, neighbors) in adj_list.iter().enumerate() {
            graph.nodes.push(GraphNode::new(i, neighbors.clone()));
        }
        graph
    }

    fn print(&self, start: usize) {
        let mut visited = HashSet::new();
        self.print_recursive(start, &mut visited);
    }

    fn print_recursive(&self, start: usize, visited: &mut HashSet<usize>) {
        if visited.contains(&start) {
            return;
        }
        visited.insert(start);
        self.nodes[start].process();
        for &neighbor in &self.nodes[start].neighbors {
            self.print_recursive(neighbor, visited);
        }
    }

    // There are no pointer relationships, so deep copy can be very simple.
    fn deep_copy(&self) -> Graph {
        let copied_nodes = self.nodes
            .iter()
            .map(|node| GraphNode::new(node.val, node.neighbors.clone()))
            .collect();

        Graph { nodes: copied_nodes }
    }
}

struct GraphNode {
    val: usize,
    neighbors: Vec<usize>,
}

impl GraphNode {
    fn new(val: usize, neighbors: Vec<usize>) -> Self {
        Self { val, neighbors }
    }

    fn process(&self) {
        println!("Processing node {}", self.val);
    }
}

fn main() {
    //     0    
    //   / | \
    //  3  |  1
    //   \ |
    //     2
    //   /
    //  4
    let adjacency_list = [
        vec![1, 2, 3],
        vec![0],
        vec![0, 3, 4],
        vec![0, 2],
        vec![2],
    ];

    let graph = Graph::from_adjacency_list(&adjacency_list);

    println!("DFS from node graph[0]:");
    graph.print(0);

    let graph_copy = graph.deep_copy();

    println!("DFS from node graph_copy[0]:");
    graph_copy.print(0);
}

## V2
* Traverse the graph (DFS)
* Duplicate nodes while traversing
* Use a hashmap where original nodes are the key and the corresponding cloned node is the value

In [ ]:
use std::collections::{HashMap, HashSet};

struct Graph {
    nodes: Vec<GraphNode>,
}

impl Graph {
    fn new() -> Self {
        Self { nodes: Vec::new() }
    }

    fn from_adjacency_list(adj_list: &[Vec<usize>]) -> Self {
        let mut graph = Graph::new();
        for (i, neighbors) in adj_list.iter().enumerate() {
            graph.nodes.push(GraphNode::new(i, neighbors.clone()));
        }
        graph
    }

    fn print(&self, start: usize) {
        let mut visited = HashSet::new();
        self.dfs_recursive(start, &mut visited);
    }

    fn dfs_recursive(&self, start: usize, visited: &mut HashSet<usize>) {
        if visited.contains(&start) {
            return;
        }
        visited.insert(start);
        self.nodes[start].process();
        for &neighbor in &self.nodes[start].adjacent_indices {
            self.dfs_recursive(neighbor, visited);
        }
    }

    fn deep_copy(&self) -> Graph {
        if self.nodes.is_empty() {
            return Graph::new();
        }

        let mut clone_map = HashMap::new();
        let _ = self.deep_copy_recursive(0, &mut clone_map);

        let mut nodes = vec![GraphNode::new(0, vec![]); self.nodes.len()];
        for (i, node) in clone_map.into_iter() {
            nodes[i] = node;
        }

        Graph { nodes }
    }

    fn deep_copy_recursive(&self, index: usize, clone_map: &mut HashMap<usize, GraphNode>) -> GraphNode {
        if let Some(existing) = clone_map.get(&index) {
            return existing.clone();
        }

        let node = &self.nodes[index];
        let mut cloned_node = GraphNode::new(node.val, vec![]);
        clone_map.insert(index, cloned_node.clone());

        for &neighbor_index in &node.adjacent_indices {
            let cloned_neighbor = self.deep_copy_recursive(neighbor_index, clone_map);
            cloned_node.adjacent_indices.push(cloned_neighbor.val);
        }

        clone_map.insert(index, cloned_node.clone());
        cloned_node
    }
}

#[derive(Clone)]
struct GraphNode {
    val: usize,
    adjacent_indices: Vec<usize>,
}

impl GraphNode {
    fn new(val: usize, adjacent_indices: Vec<usize>) -> Self {
        Self {
            val,
            adjacent_indices,
        }
    }

    fn process(&self) {
        println!("Processing node {}", self.val);
    }
}

fn main() {
    //     0
    //   / | \
    //  3  |  1
    //   \ |
    //     2
    //   /
    //  4
    let adjacency_list = [vec![1, 2, 3], vec![0], vec![0, 3, 4], vec![0, 2], vec![2]];

    let graph = Graph::from_adjacency_list(&adjacency_list);

    println!("DFS from node graph[0]:");
    graph.print(0);

    let graph_copy = graph.deep_copy();

    println!("DFS from node graph_copy[0]:");
    graph_copy.print(0);
}
